In [1]:
import camelot
import json
import os
import re

In [2]:
def get_continued_tables(tables, threshold):

    continued_tables = {}
    previous_table = False
    group_counter = 0

    # typical height of a pdf is 842 points and bottom margins are anywhere between 56 and 85 points
    # therefore, accounting for margins, 792
    page_height = 792

    # iterate over the tables
    for i, table in enumerate(tables):

        # if a previous table exists (remember, we start with this as false)
        # and the previous table was on the previous page
        # and the number of columns of both tables is the same
        if previous_table and table.page == previous_table.page + 1 and len(table.cols) == len(previous_table.cols):

            # get the bottom coordinate of the previous table
            # note that for pdfs the origin (0, 0) typically starts from the bottom-left corner of the page,
            # with the y-coordinate increasing as you move upwards
            # this is why for {x0, y0, x1, y1} we need the y0 as the bottom
            previous_table_bottom = previous_table._bbox[1]

            # get the top coordinate of the current table
            # for {x0, y0, x1, y1} we need the y1 as the top
            current_table_top = table._bbox[3]

            # if the previous table ends in the last 15% of the page and the current table starts in the first 15% of the page
            if previous_table_bottom < (threshold / 100) * page_height and current_table_top > (1 - threshold / 100) * page_height:

                # if we don't have started this group of tables
                if (continued_tables.get(group_counter) is None):

                    # start by adding the first table
                    continued_tables[group_counter] = [previous_table]

                # add any of the sunsequent tables to the group
                continued_tables[group_counter].append(table)

            # if this is not a continuation of the previous table
            else:

                # increment the group number
                group_counter += 1;

        # if this is not a continuation of the previous table
        else:

            # increment the group number
            group_counter += 1;

        # the current table becomes the previous table for the next iteration
        previous_table = table

    # transform the dictionary into an array of arrays
    continued_tables = [value for value in continued_tables.values()]

    # return the combined tables
    return continued_tables

In [3]:
def table_to_json(table_data, table_info):
    """Convert table data to JSON format"""
    if not table_data:
        return {}
    
    # Create JSON structure
    json_data = {
        "metadata": {
            "source_file": table_info["source_file"],
            "page": table_info["page"],
            "table_order": table_info["order"],
            "total_rows": len(table_data),
            "total_columns": len(table_data[0]) if table_data else 0
        },
        "headers": [],
        "data": []
    }
    
    # Add headers (first row)
    if len(table_data) > 0:
        headers = [str(cell).strip() for cell in table_data[0]]
        
        # Replace first 3 headers with fixed names
        if len(headers) >= 1:
            headers[0] = "STT"
        if len(headers) >= 2:
            headers[1] = "hang_hoa"
        if len(headers) >= 3:
            headers[2] = "yeu_cau_ky_thuat"
            
        json_data["headers"] = headers
        
        # Add data rows (skip header)
        for i, row in enumerate(table_data[1:], 1):
            row_dict = {}
            for j, cell in enumerate(row):
                # Use header as key, fallback to column index if header is empty
                key = json_data["headers"][j] if j < len(json_data["headers"]) and json_data["headers"][j] else f"column_{j}"
                row_dict[key] = str(cell).strip()
            
            json_data["data"].append({
                "row_index": i,
                "values": row_dict
            })
    
    return json_data

In [4]:
def get_biggest_table(pdf_path, threshold):
    tables = camelot.read_pdf(pdf_path, flavor = 'lattice', pages = 'all')
    continued_tables = get_continued_tables(tables, threshold)

    # get the name of the PDF file we are processing (without the extension)
    pdf_file_name = os.path.splitext(os.path.basename(pdf_path))[0]

    processed = []
    all_table_jsons = []

    # iterate over found tables
    for i, table in enumerate(tables):

        # if table was already processed as part of a group
        if table in processed: continue

        # check if the current table is a continued table
        is_continued = any(table in sublist for sublist in continued_tables)

        # collect all table data (current table + continued tables if any)
        all_table_data = list(table.data)

        # if the current table is a continued table, append all subsequent continued tables data
        if is_continued:

            # get the index of the group in "continued_tables" associated with the current table
            group_index = next(index for index, sublist in enumerate(continued_tables) if table in sublist)

            # iterate over the tables in said group and append their data
            for continued_table in continued_tables[group_index]:

                # skip the current table as it's already added
                if continued_table == table or continued_table in processed: continue

                # append the data of the continued table (skip header for subsequent tables)
                all_table_data.extend(continued_table.data[1:] if len(continued_table.data) > 1 else [])

                # keep track of processed tables
                processed.append(continued_table)

        # convert to JSON
        table_info = {
            "source_file": pdf_file_name,
            "page": table.parsing_report['page'],
            "order": table.parsing_report['order']
        }
        
        json_data = table_to_json(all_table_data, table_info)
        all_table_jsons.append(json_data)
        
        # mark current table as processed
        processed.append(table)

    # find the table with the most rows
    if all_table_jsons:
        largest_table = max(all_table_jsons, key=lambda x: x.get('metadata', {}).get('total_rows', 0))
        
        # return the JSON of the largest table
        print(json.dumps(largest_table, ensure_ascii=False, indent=2))
        return largest_table
    else:
        print("No tables found in the PDF.")
        return None

In [ ]:
hello = get_biggest_table("D:/study/LammaIndex/documents/Chuong_V_Yeu_cau_ky_thuat.pdf",15)

In [6]:
data = hello["data"]

In [7]:
import uuid
def clean_text(text):
    """Làm sạch text, loại bỏ ký tự xuống dòng thừa"""
    return re.sub(r'\n+', '', text.strip())

def split_requirements(text):
    """Tách các yêu cầu dựa trên dấu gạch đầu dòng"""
    requirements = []
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('- '):
            requirements.append(line[2:].strip())
        elif line and not any(line.startswith(prefix) for prefix in ['- ']):
            if requirements:
                requirements[-1] += ' ' + line
            else:
                requirements.append(line)
    return requirements

def generate_random_key():
    """Tạo key random 5 ký tự từ UUID"""
    return str(uuid.uuid4()).replace('-', '')[:5].upper()

def convert_to_new_format(data):
    result = []
    current_product = None
    current_category = None
    
    for item in data:
        values = item['values']
        stt_raw  = values['STT']
        hang_hoa = clean_text(values['hang_hoa'])
        yeu_cau = values['yeu_cau_ky_thuat']


        stt = stt_raw.strip()

        roman_pattern = r'^(VII|VIII|IX|X|XI|XII|I{1,3}|IV|V|VI)\s+(.+)'
        roman_match = re.match(roman_pattern, stt)
        # Nếu STT là số La Mã (I, II, III...) thì đây là tên sản phẩm
        hang_hoa_roman_match = re.match(roman_pattern, hang_hoa)
        if roman_match and not hang_hoa and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = roman_match.group(1)  # Số La Mã
            product_name = roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None
        elif hang_hoa_roman_match and not stt_raw and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = hang_hoa_roman_match.group(1)  # Số La Mã
            product_name = hang_hoa_roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None        
        
        elif stt in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'XIV']:
            if current_product:
                result.append(current_product)
            
            current_product = {
                "ten_san_pham": hang_hoa,
                "cac_muc": []
            }
            current_category = None
            
        # Nếu STT là số (1, 2, 3...) thì đây là danh mục
        elif stt.isdigit():
            current_category = {
                "ten_hang_hoa": hang_hoa,
                "thong_so_ky_thuat": {}
            }
            
            # Xử lý yêu cầu kỹ thuật cho danh mục
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                for req in requirements:
                    key = generate_random_key()  # Tạo key random
                    current_category["thong_so_ky_thuat"][key] = clean_text(req)
            if current_product:
                current_product["cac_muc"].append(current_category)
                
        # Nếu STT trống thì đây là thông số kỹ thuật chi tiết
        elif stt == '' and current_category and hang_hoa:
            # Tạo key random cho thông số kỹ thuật
            key = generate_random_key()
            
            # Làm sạch tên hàng hóa và yêu cầu kỹ thuật
            clean_hang_hoa = clean_text(hang_hoa)
            clean_yeu_cau = clean_text(yeu_cau)
            
            current_category["thong_so_ky_thuat"][key] = [clean_hang_hoa, clean_yeu_cau]
        elif stt == '' and current_category and not hang_hoa:
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                
                # Lấy key cuối cùng trong thong_so_ky_thuat (nếu có)
                existing_keys = list(current_category["thong_so_ky_thuat"].keys())
                last_key = existing_keys[-1] if existing_keys else None
                
                for req in requirements:
                    clean_req = clean_text(req)
                    
                    # Kiểm tra chữ cái đầu có viết hoa HOẶC có gạch đầu dòng không
                    has_dash = req.strip().startswith('- ')
                    has_uppercase = clean_req and clean_req[0].isupper()
                    
                    if has_uppercase or has_dash:
                        # Chữ đầu viết hoa HOẶC có gạch đầu dòng -> tạo key mới
                        key = generate_random_key()
                        current_category["thong_so_ky_thuat"][key] = clean_req
                        last_key = key
                    else:
                        # Chữ đầu không viết hoa VÀ không có gạch đầu dòng -> nối vào key trước đó
                        if last_key and last_key in current_category["thong_so_ky_thuat"]:
                            current_category["thong_so_ky_thuat"][last_key] += " " + clean_req
                        else:
                            # Nếu không có key trước đó thì vẫn tạo key mới
                            key = generate_random_key()
                            current_category["thong_so_ky_thuat"][key] = clean_req
                            last_key = key
    
    # Thêm sản phẩm cuối cùng
    if current_product:
        result.append(current_product)
    
    return result

# Chuyển đổi dữ liệu
converted_data = convert_to_new_format(data)

In [11]:
converted_data

[{'ten_san_pham': 'Tải giả xả acquy',
  'cac_muc': [{'ten_hang_hoa': 'Yêu cầu chung',
    'thong_so_ky_thuat': {'BE731': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
     'EC01D': 'Thiết bị mới 100% chưa qua sử dụng',
     '6AB3B': 'Thiết bị phải được sản xuất từ năm 2021 trở lại đây',
     'EC972': 'Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản xuất, tối thiểu 12 tháng.'}},
   {'ten_hang_hoa': 'Thông số kỹ thuật',
    'thong_so_ky_thuat': {'E519F': ['Điện trở tiếp địa', '< 0.1 Ohm'],
     '9E6C5': ['Độ ồn', '< 75 dB khi hoạt động hết công suất.'],
     '770CB': ['Điều kiện nhiệt độ',
      '-  Dải nhiệt độ hoạt động: 0 ÷ 40°C,  -  Dải nhiệt độ lưu kho: -25°C ÷70°C.'],
     '2E748': ['Điều kiện độ ẩm',
      '-  Độ ẩm tương đối từ 5 ÷ 95%, không đọng sương.'],
     'CBDD6': ['Điện áp acquy cao nhất – tính theo giá trị tuyệt đối',
      '≥ 270 V'],
     '7C066': ['Điện áp acquy nhỏ nhất', '≤ 18 V'],
    

In [12]:
context_queries = {}  # Đổi từ list sang dict
product_key = {}
for item in converted_data:
    ten_san_pham = item['ten_san_pham']
    for muc in item['cac_muc']:
        ten_hang_hoa = muc['ten_hang_hoa']
        thong_so_ky_thuat = muc['thong_so_ky_thuat']
        for key, value in thong_so_ky_thuat.items():
            if isinstance(value, list):
                value_str = ' '.join(value)
            else:
                value_str = value
            context_queries[key] = {
                "ten_san_pham": ten_san_pham,
                "ten_hang_hoa": ten_hang_hoa,
                "value": value_str
            }
            if ten_san_pham not in product_key:
                product_key[ten_san_pham] = []
            product_key[ten_san_pham].append(key)


In [13]:
context_queries

{'BE731': {'ten_san_pham': 'Tải giả xả acquy',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ, có chứng nhận chất lượng sản phẩm của nhà sản xuất.'},
 'EC01D': {'ten_san_pham': 'Tải giả xả acquy',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thiết bị mới 100% chưa qua sử dụng'},
 '6AB3B': {'ten_san_pham': 'Tải giả xả acquy',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thiết bị phải được sản xuất từ năm 2021 trở lại đây'},
 'EC972': {'ten_san_pham': 'Tải giả xả acquy',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản xuất, tối thiểu 12 tháng.'},
 'E519F': {'ten_san_pham': 'Tải giả xả acquy',
  'ten_hang_hoa': 'Thông số kỹ thuật',
  'value': 'Điện trở tiếp địa < 0.1 Ohm'},
 '9E6C5': {'ten_san_pham': 'Tải giả xả acquy',
  'ten_hang_hoa': 'Thông số kỹ thuật',
  'value': 'Độ ồn < 75 dB khi hoạt động hết công suất.'},
 '770CB': {'ten_san_pham': 'Tải giả xả acquy',
  'ten_

In [14]:
product_key

{'Tải giả xả acquy': ['BE731',
  'EC01D',
  '6AB3B',
  'EC972',
  'E519F',
  '9E6C5',
  '770CB',
  '2E748',
  'CBDD6',
  '7C066',
  'B0712',
  '86643',
  '125E5',
  'F8A7A',
  'B6C3C',
  'CF71A',
  '63F35',
  '3338C',
  'A20DC',
  'E74E6',
  '047B9',
  '5511D',
  '59C56',
  '0D075',
  'DC9CA',
  '544FD',
  'BAB75',
  '57B6D',
  '501EC',
  '8031A',
  '2FB84',
  '387C2',
  '8B098',
  '1D9B3',
  '16A80',
  '28A66',
  'F1CE0',
  '0F01C',
  'D3ADB',
  'C8089',
  'B1986',
  '3636F',
  'EF25D',
  '822B0'],
 'Đồng hồ đo nội trở acquy': ['CF5D4',
  'EB45C',
  '024F2',
  'FA7EB',
  '56F39',
  'A17B1',
  '211ED',
  '7F231',
  'F5B33',
  '7A123',
  'CD1D1',
  'B6977',
  'A2F9F',
  'BAB82',
  'BDAE5',
  '67502',
  'CA1CA',
  'E1D0F',
  '10A8E',
  'EB439'],
 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': ['C83D9',
  '0C675',
  '3A686',
  'F6AB1',
  '92E16',
  '97267',
  '263C3',
  'D67CA',
  '2B54F',
  '182CE',
  '660BA',
  '41B55',
  '67B34',
  '9FBD4',
  '175E8',
  'ACD58',
  '

In [ ]:
# Lưu với tên file mặc định
with open('context_prompts.json', 'w', encoding='utf-8') as f:
    json.dump(context_prompts, f, ensure_ascii=False, indent=2)

print(f"✅ Đã lưu {len(context_prompts)} context prompts vào file: context_prompts.json")

In [15]:
def prompt_create_query(context_prompt):
    prompt =  f"""
    Bạn sẽ nhận được một đoạn mô tả kỹ thuật ngắn gọn (context_prompt), thường là các mảnh thông tin kỹ thuật rời rạc. Nhiệm vụ của bạn là chuyển đổi đoạn mô tả đó thành một truy vấn (query) hoàn chỉnh bằng ngôn ngữ tự nhiên, với mục tiêu:

- Diễn đạt lại đoạn mô tả dưới dạng một câu hỏi hoặc câu truy vấn rõ ràng, dễ hiểu.
- Giữ nguyên đầy đủ tất cả các thông tin kỹ thuật có trong câu gốc (chế độ, dòng điện, điện áp, bước công suất,...).
- Không được loại bỏ hoặc làm mờ bất kỳ chi tiết kỹ thuật nào.
- Truy vấn cần phù hợp để tìm kiếm thông tin tương đồng về mặt ngữ nghĩa trong một hệ thống truy xuất dữ liệu kỹ thuật.

Dưới đây là các ví dụ:

**Input:** "Tải giả xả acquy Thông số kỹ thuật Dòng xả lớn nhất với dải điện áp 220V DC ÷ 240V DC – tương ứng với 1 bộ ≥ 70 A"

**Output:** "Dòng xả lớn nhất của tải giả xả acquy là bao nhiêu khi dải điện áp nằm trong khoảng 220V DC đến 240V DC, và tương ứng với 1 bộ thì có đạt dòng xả ≥ 70A không?"

---
**Input:** "Tải giả xả acquy Thông số kỹ thuật Các chế độ xả - Gồm 4 chế độ: dòng không đổi, công suất không đổi, theo đặc tính dòng cho trước, điều chỉnh thủ công."

**Output:** "Tải giả xả acquy hỗ trợ những chế độ xả nào? Có phải gồm dòng không đổi, công suất không đổi, theo đặc tính dòng định trước, và điều chỉnh thủ công không?"

---
**Input:** "Tải giả xả acquy Thông số kỹ thuật Bước công suất điều chỉnh xả tải 100 W"

**Output:** "Bước điều chỉnh công suất xả tải nhỏ nhất của tải giả xả acquy là bao nhiêu? Có phải là 100W không?"

---
Trả lời từng `context_prompt` theo cấu trúc như trên.


    DỮ LIỆU ĐẦU VÀO:
    ---
    {context_prompt}
    ---

    CHỈ CẦN TRẢ VỀ CÂU TRUY VẤN.
    """
    return prompt

In [16]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [17]:
test = dict(list(context_queries.items())[70:75])
test

{'263C3': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.'},
 'D67CA': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4'},
 '2B54F': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Công suất mỗi module chỉnh lưu: ≥ 3000W'},
 '182CE': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥ 3 module'},
 '660BA': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Attomat DC: + PL (

In [18]:
for key in test:
    data = test[key] 
    prompt = prompt_create_query(f"{data['ten_san_pham']} {data['ten_hang_hoa']} {data['value']}")
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        temperature=0
    )
    output_text = response.output_text.strip()
    test[key]["query"]=output_text

In [ ]:
test, product_key

In [20]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import VectorStoreInfo
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)

# Cấu hình LLM và Embedding
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = OpenAI(model="gpt-4o-mini")
# Cấu hình client Qdrant
client = QdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
# Khởi tạo Vector Store
vector_store = QdrantVectorStore(
    collection_name="thong_tin_san_pham",
    client=client,
    aclient=aclient,
)

def retrieve_document(query_str):
    file_names = []
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_document = MetadataFilters(
        filters=[
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="summary_document"),
        ],
    condition=FilterCondition.AND,
    )
    retriever_document = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_document)
    
    results = retriever_document.retrieve(query_str)

    for result in results:
        metadata = result.metadata
        file_names.append(metadata["file_name"])

    return file_names

def retrieve_chunk(file_names, query_str):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_chunk = MetadataFilters(
        filters=[
            MetadataFilter(key="file_name", operator=FilterOperator.IN, value=file_names),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
        ],
        condition=FilterCondition.AND,
    )

    retriever_chunk = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_chunk)
    
    results = retriever_chunk.retrieve(query_str)
    content = ""
    for i, result in enumerate(results, start=1):
        metadata = result.metadata
        file_name = metadata["file_name"]+ ".pdf"
        page = metadata["page"]
        table = metadata["table_name"]
        figure_name = metadata.get("figure_name")
        text = result.text.strip()
        content += f"Chunk {i} trong file {file_name} tại trang {page}, có chứa bảng {table} và hình {figure_name} có nội dung:\n{text}\n\n"

    return content


        




D:\study\LammaIndex\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
for product in product_key:
    file_names = retrieve_document(product)
    items = product_key[product]
    for item in items:
        if item not in test:
            continue
        query = test[item]["query"]
        content = retrieve_chunk(file_names, query)
        test[item]["content"] = content

In [ ]:
test

In [23]:
SYSTEM_PROMPT = """
Bạn được cung cấp:
- Một hoặc nhiều đoạn văn bản (chunk) từ tài liệu kỹ thuật, kèm metadata: tên file, mục, bảng/hình (nếu có), số trang
- Một yêu cầu kỹ thuật cụ thể.

#Yêu cầu:
# 1. Tìm thông tin kỹ thuật liên quan trực tiếp đến yêu cầu kỹ thuật.
# 2. Trích xuất giá trị thông số để xác định khả năng đáp ứng theo yêu cầu.
# 3. Dẫn chứng rõ: file, section, table/figure name (nếu có), page, nội dung trích dẫn của những tài liệu liên quan, những tài liệu khác không liên quan thì bỏ qua.

# Output: JSON gồm các trường, trả lời bằng tiếng việt:
- yeu_cau_ky_thuat
- kha_nang_dap_ung
- tai_lieu_tham_chieu" 

# Ví dụ:
Input:
Yêu cầu: "Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4"  
Chunk: "...NetSure 731 A41-S8: 4 rectifier slots (standard), expandable to 6..."  
Metadata:  
- file: "Netsure-731-A41-user-manual.pdf"  
- section: "Table 1-1 Configuration of power system"  
- page: 2"
"""

# Định nghĩa function schema
FUNCTION_SCHEMA = {
    "name": "danh_gia_ky_thuat",
    "description": "Đánh giá khả năng đáp ứng của sản phẩm theo yêu cầu kỹ thuật từ chunk tài liệu.",
    "parameters": {
        "type": "object",
        "properties": {
            "yeu_cau_ky_thuat": {"type": "string"},
            "kha_nang_dap_ung": {"type": "string"},
            "tai_lieu_tham_chieu": {
                "type": "object",
                "properties": {
                    "file": {"type": "string"},
                    "section": {"type": "string"},
                    "table_or_figure": {"type": "string"},
                    "page": {"type": "integer"},
                    "evidence": {"type": "string"}
                },
                "required": ["file", "section", "page", "evidence"]
            }
        },
        "required": ["yeu_cau_ky_thuat", "kha_nang_dap_ung", "tai_lieu_tham_chieu"]
    }
}

In [24]:
# Tạo assistant
# === CREATE ASSISTANT ===
def create_assistant():
    assistant = client.beta.assistants.create(
        name="Technical Document Evaluator",
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === CREATE THREAD ===
def create_thread():
    thread = client.beta.threads.create()
    return thread.id

# === UPDATE ASSISTANT ===
def update_assistant(assistant_id):
    assistant = client.beta.assistants.update(
        assistant_id=assistant_id,
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === EVALUATE TECHNICAL REQUIREMENT ===
def evaluate_technical_requirement(user_prompt, assistant_id):
    # 1. Tạo thread riêng cho mỗi lần gọi
    thread = client.beta.threads.create()
    thread_id = thread.id

    # 2. Gửi message vào thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=user_prompt
    )

    # 3. Tạo run
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )

    # 4. Chờ assistant xử lý (tối đa 20s)
    for _ in range(20):
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        if run.status not in ["queued", "in_progress"]:
            break
        time.sleep(1)

    # 5. Lấy arguments trực tiếp
    if run.status == "requires_action":
        call = run.required_action.submit_tool_outputs.tool_calls[0]
        print(f"👉 Assistant đã gọi tool: {call.function.name}")
        print("🧠 Dữ liệu JSON assistant muốn trả về:")
        print(call.function.arguments)
        return call.function.arguments

    elif run.status == "completed":
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        for msg in messages.data:
            print(f"[{msg.role}] {msg.content[0].text.value}")
        return None

    else:
        print(f"Run status: {run.status}")
        return None



In [ ]:
test

In [26]:
import openai
from openai import OpenAI
import json
client = OpenAI()

In [28]:
# Ví dụ sử dụng
import time
assistant_id = create_assistant()
print(f"Assistant ID: {assistant_id}")


# Ví dụ user prompt

for key in test:
    if not test[key].get('content'):
        continue
    data = test[key]
    value = data['value']
    content = data['content']
    # test[key].pop("content", None)
    
    user_prompt = f'''
    Yêu cầu: {value}
    Chunk và metadata: {content}
    '''
    
    # Gọi hàm đánh giá với thread riêng
    result = evaluate_technical_requirement(user_prompt, assistant_id)
    test[key]['response'] = result




Assistant ID: asst_t8fWR9otgSl4YtP0d3FNwb52


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15484\985798817.py:30: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15484\985798817.py:34: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15484\985798817.py:41: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15484\985798817.py:49: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)


👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.","kha_nang_dap_ung":"Không có thông tin về thời gian bảo hành trong tài liệu cung cấp.","tai_lieu_tham_chieu":{"file":"Converter_Brochure.pdf","section":"","table_or_figure":"","page":2,"evidence":""}}
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4","kha_nang_dap_ung":"Đáp ứng, sản phẩm có tối đa 4 khe cắm module chỉnh lưu.","tai_lieu_tham_chieu":{"file":"NetSure_732_User_Manual.pdf","section":"Table 1-1 Configuration of power system","table_or_figure":"Table 1-1","page":8,"evidence":"Maximum configuration: 4 pieces"}}
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Công suất mỗi module chỉnh lưu: ≥ 3000W","kha_nang_dap_ung":"3000W Maximum","tai_lieu_tham_

In [29]:
test

{'263C3': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.',
  'query': 'Bộ chuyển đổi nguồn 220VAC/48VDC kèm theo 02 dàn acquy 200Ah có thời gian bảo hành tối thiểu là bao lâu? Có phải là theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng không?',
  'content': 'Chunk 1 trong file Converter_Brochure.pdf tại trang 2, có chứa bảng DC Output Specifications và hình None có nội dung:\n### DC Output\n\n| Output voltage, Adjustment Range        | -42 to -58VDC                                                                   |\n| --------------------------------------- | ------------------------------------------------------------------------------- |\n| Output Power                            | 3000W Maximum                                                                   |\n| Output Power, derated for Input Voltage | See diagram      